In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
"""
%%capture
zip_path = '/content/drive/MyDrive/GenClass/temp/modelset.zip'
extract_folder = '/content/drive/MyDrive/GenClass/temp/'

!unzip -o "{zip_path}" -d "{extract_folder}"
"""
!pip install wordninja

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541530 sha256=b5c89844a55bcf1bf6688c47a4971775f060da56354aa837b4e66846b697fbf9
  Stored in directory: /root/.cache/pip/wheels/e6/66/9c/712044a983337f5d44f90abcd244bd4b8ad28ee64750404b50
Successfully built wordninja


In [ ]:
import xml.etree.ElementTree as ET
import json
import re
import urllib.parse
import unicodedata
import wordninja
class XMI2AST:
    primitive_types = {"string", "boolean", "int", "float", "double", "long", "short", "byte", "char"}

    @staticmethod
    def normalize_type(raw_type: str) -> str:
        if not raw_type:
            return ""

        # Decode URL-encoded characters
        raw_type = urllib.parse.unquote(raw_type)

        # Remove leading/trailing whitespace
        raw_type = raw_type.strip()

        # Ignore internal reference types that start with "_"
        if raw_type.startswith("_"):
            return ""

        # Remove XMI href parts (e.g., path#//Type)
        if '//' in raw_type:
            raw_type = raw_type.split('//')[-1]

        # Normalize known variations
        normalization_map = {
            "integer": "int",
            "bool": "boolean",
            "boolean": "boolean",
            "string": "string",
            "float": "float",
            "double": "double",
            "long": "long",
            "short": "short",
            "byte": "byte",
            "char": "char"
        }

        lower_type = raw_type.lower()
        if lower_type in normalization_map:
            return normalization_map[lower_type]

        return raw_type[0].upper() + raw_type[1:]


    @staticmethod
    def to_pascal_case(name: str) -> str:
        if not name:
            return ""

        # First split on delimiters like space, underscore, dash
        parts = re.split(r'[\s_-]+', name.strip())

        # If splitting didn't produce multiple parts, use wordninja
        if len(parts) == 1:
            parts = wordninja.split(name)

        return ''.join(word.capitalize() for word in parts if word)


    @staticmethod
    def to_camel_case(name: str) -> str:
        if not name:
            return ""

        # First try splitting by delimiters (_ - space)
        parts = re.split(r'[\s_-]+', name.strip())

        # If splitting fails (i.e., only one part and all lowercase), use wordninja
        if len(parts) == 1 and parts[0].islower():
            parts = wordninja.split(name.strip())

        if not parts:
            return ""

        return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:])

    @staticmethod
    def lower_first_letter(s: str) -> str:
        if not s:
            return s
        return s[0].lower() + s[1:]


    @staticmethod
    def extract_visibility(visibility):
        mapping = {
            "public": "+",
            "private": "-",
            "protected": "#",
            "package": "~"
        }
        return mapping.get(visibility, "+")

    def is_pascal_case(s: str) -> bool:
        """
        Returns True if the input string follows PascalCase naming.
        """
        pattern = r'^[A-Z][a-z0-9]*(?:[A-Z][a-z0-9]*)*$'
        return bool(re.match(pattern, s))

    @staticmethod
    def replace_and_or(s: str) -> str:
        # Step 1: Strip / or & at the start or end
        s = s.strip('/&,')

        # Step 2: Split by / or & inside the string
        parts = re.split(r'([/&,])', s)

        # Step 3: Rebuild with replacements
        result_parts = []
        i = 0
        while i < len(parts):
            part = parts[i]
            if part == '/':
                result_parts.append("Or")
                i += 1
            elif part == '&' or part == ',':
                result_parts.append("And")
                i += 1
            else:
                # Normal string part — capitalize first letter, keep rest as is
                cleaned = part.strip()
                if cleaned:
                    result_parts.append(cleaned[0] + cleaned[1:])
                i += 1

        # Join all parts
        return ''.join(result_parts)


    @staticmethod
    def is_camel_case(s: str) -> bool:
        """
        Returns True if the input string follows camelCase naming.
        """
        pattern = r'^[a-z0-9]+(?:[A-Z][a-z0-9]*)*$'
        return bool(re.match(pattern, s))

    @staticmethod
    def remove_after_char(s: str, specific_char) -> str:
        return s.split(specific_char, 1)[0]

    @staticmethod
    def remove_accents(text: str) -> str:
        # Normalize string to decomposed form (NFKD)
        nfkd_form = unicodedata.normalize('NFKD', text)
        # Filter out combining characters (accents)
        return ''.join(c for c in nfkd_form if not unicodedata.combining(c))


    @staticmethod
    def extract_after_last_dot_if_java(s: str) -> str:
        if s.startswith("Java"):
            return s.split(".")[-1]
        return s

    @staticmethod
    def remove_after_brackets_and_parentheses(s: str) -> str:
        # Step 1: Remove everything after and including the first '['
        if '[' in s:
            s = s.split('[')[0]

        # Step 2: Remove everything after and including the first '('
        if '(' in s:
            s = s.split('(')[0]

        return s

    @staticmethod
    def replace_part(s: str, replaced: str, replacement: str = "") -> str:
        return s.replace(replaced, replacement)

    @staticmethod
    def extract_generic_type(s: str) -> str:
        match = re.search(r'<\s*([a-zA-Z0-9_]+)\s*>', s)
        if match:
            return match.group(1)
        return s  # return original if no angle brackets found

    def extract_inner_generic(s: str) -> str:
        match = re.search(r'<\s*([^<>]+?)\s*>', s)
        if match:
            return match.group(1).strip()
        return s  # Return original if no <...> found

    @staticmethod
    def split_method_and_return(sig: str) -> tuple[str, str]:
        """
        Given a signature like "getName():string",
        returns ("getName()", "string").
        If no return type is present, returns (sig, "").
        """
        sig = sig.strip()
        # Regex explanation:
        # ^(.+\(\))   → match name() part (group 1)
        # (?:\s*:\s*(.+))? → optional :returnType (group 2)
        pattern = r'^(.+\(\))(?:\s*:\s*(.+))?$'
        match = re.match(pattern, sig)
        if match:
            method = match.group(1)
            ret_type = match.group(2) or ""
            return method, ret_type
        return sig, ""


    @staticmethod
    def manipulate_attr_name(attr_name):
        if attr_name is None:
            return ""
        attr_name=attr_name.strip()
        attr_name = XMI2AST.extract_generic_type(attr_name)
        attr_name = XMI2AST.replace_and_or(attr_name)
        attr_name= XMI2AST.remove_after_brackets_and_parentheses(attr_name)
        attr_name= XMI2AST.remove_after_char(attr_name,'.')
        attr_name= XMI2AST.remove_after_char(attr_name,'=')
        attr_name = XMI2AST.lower_first_letter(attr_name)
        attr_name = XMI2AST.remove_accents(attr_name)
        attr_name = XMI2AST.remove_after_char(attr_name,"*")

        attr_name = XMI2AST.replace_part(attr_name,"+")
        attr_name = XMI2AST.replace_part(attr_name,"#")
        attr_name = XMI2AST.replace_part(attr_name,":")
        attr_name = XMI2AST.replace_part(attr_name,"$")
        attr_name = XMI2AST.replace_part(attr_name,"@")
        attr_name = XMI2AST.replace_part(attr_name,'\\')
        attr_name = XMI2AST.replace_part(attr_name,"'s")
        attr_name = XMI2AST.replace_part(attr_name,">")
        attr_name = XMI2AST.replace_part(attr_name,"<>")
        attr_name = XMI2AST.replace_part(attr_name,"'")
        attr_name = XMI2AST.replace_part(attr_name,"-")
        attr_name = XMI2AST.replace_part(attr_name,"l'")
        attr_name = XMI2AST.replace_part(attr_name,"_")
        attr_name = XMI2AST.to_camel_case(attr_name.lower())
        return attr_name

    @staticmethod
    def manipulate_method_name(op_name):
        if op_name is None:
            return ""
        op_name=op_name.strip()
        op_name = XMI2AST.remove_accents(op_name)
        op_name = XMI2AST.replace_and_or(op_name.strip())
        op_name= XMI2AST.remove_after_char(op_name,'.')
        op_name= XMI2AST.remove_after_char(op_name,'(')
        op_name = XMI2AST.replace_part(op_name,"void:")
        op_name = XMI2AST.replace_part(op_name,"+void")
        op_name = XMI2AST.replace_part(op_name,"+card")
        op_name = XMI2AST.replace_part(op_name,"+bool")
        op_name = XMI2AST.replace_part(op_name,"~")
        op_name = XMI2AST.replace_part(op_name,"+")
        op_name = XMI2AST.replace_part(op_name,"s'")
        op_name = XMI2AST.replace_part(op_name,"_")
        op_name = XMI2AST.replace_part(op_name,":")
        op_name = XMI2AST.replace_part(op_name,"'s")
        op_name = XMI2AST.replace_part(op_name,"*")
        op_name = XMI2AST.replace_part(op_name,"<<property>>")
        op_name = XMI2AST.replace_part(op_name,"-")
        op_name = XMI2AST.to_camel_case(op_name.lower())
        return  op_name

    @staticmethod
    def manipulate_class_name(class_name):
        if class_name is None:
            return ""
        class_name=class_name.strip()
        class_name = XMI2AST.remove_accents(class_name)
        class_name = XMI2AST.extract_after_last_dot_if_java(class_name)
        class_name = XMI2AST.replace_part(class_name,"+")
        class_name = XMI2AST.replace_part(class_name,".")
        class_name = XMI2AST.replace_part(class_name,"&")
        class_name = XMI2AST.replace_part(class_name,"*")
        class_name = XMI2AST.extract_inner_generic(class_name)
        class_name= XMI2AST.remove_after_brackets_and_parentheses(class_name)

        class_name = XMI2AST.replace_part(class_name,"'s")
        class_name = XMI2AST.remove_after_char(class_name,"/")
        class_name = XMI2AST.remove_after_char(class_name,":")
        class_name = XMI2AST.remove_after_char(class_name,"`")
        class_name = XMI2AST.replace_part(class_name,",")
        class_name = XMI2AST.remove_after_char(class_name,"^")
        class_name = XMI2AST.remove_after_char(class_name,"=")
        class_name = XMI2AST.to_pascal_case(class_name.lower())
        return  class_name


    @staticmethod
    def split_method_signature(sig: str) -> tuple[str, str]:
        """
        Splits a method signature "name(params)" into ("name", "(params)").
        Returns (name, params_with_parentheses) or (sig, "") if no parentheses found.
        """
        sig = sig.strip()
        match = re.match(r'^([^(]+)(\(.+\))$', sig)
        if match:
            method_name = match.group(1).strip()
            params = match.group(2).strip()
            return method_name, params
        return sig, ""



    @staticmethod
    def extract_classes(xmi_file, file_index):
        tree = ET.parse(xmi_file)
        root = tree.getroot()

        ns = {
            "xmi": "http://schema.omg.org/spec/XMI/2.1",
            "uml": "http://www.eclipse.org/uml2/5.0.0/UML",
            "xsi": "http://www.w3.org/2001/XMLSchema-instance"
        }

        classes = []

        for class_elem in root.findall(".//packagedElement[@xsi:type='uml:Class']", ns):
            raw_class_name = class_elem.attrib.get("name", "")
            class_name = XMI2AST.manipulate_class_name(raw_class_name.strip())

            if not XMI2AST.is_pascal_case(class_name) and len(class_name.strip()) > 0:
                print("Error here class: " + class_name +" "+ repr(class_name)+ " file index: " + str(file_index))
            class_node = {
                "type": "class",
                "value": class_name,
                "children": []
            }

            # Attributes
            for attr in class_elem.findall("ownedAttribute", ns):
                raw_attr_name = attr.attrib.get("name", "")
                attr_name = XMI2AST.manipulate_attr_name(raw_attr_name)
                attr_visibility_raw = attr.attrib.get("visibility")
                attr_visibility = XMI2AST.extract_visibility(attr_visibility_raw)

                attr_type = ""
                type_elem = attr.find("type", ns)
                if type_elem is not None:
                    raw_type = type_elem.attrib.get('href', "") or type_elem.attrib.get('name', "")
                    attr_type = XMI2AST.normalize_type(raw_type)
                if not attr_type and "type" in attr.attrib:
                    attr_type = XMI2AST.normalize_type(attr.attrib["type"])

                if not XMI2AST.is_camel_case(attr_name) and len(attr_name.strip()) > 0:
                    print("Error here attribute: " + attr_name +" "+ repr(attr_name)+ " file index: " + str(file_index))



                attribute_node = {
                    "type": "attribute",
                    "value": attr_name,
                    "visibility": attr_visibility,
                    "data_type": attr_type
                }
                if len(attr_name.strip()) > 0:
                  class_node["children"].append(attribute_node)

            # Methods
            for op in class_elem.findall("ownedOperation", ns):
                raw_op_name = op.attrib.get("name", "")
                op_name, detected_return_type = XMI2AST.split_method_and_return(raw_op_name.strip())
                op_name, params = XMI2AST.split_method_signature(op_name.strip())
                op_name = XMI2AST.manipulate_method_name(op_name)
                op_visibility_raw = op.attrib.get("visibility")
                op_visibility = XMI2AST.extract_visibility(op_visibility_raw)

                return_type = "void"
                for param in op.findall("ownedParameter", ns):
                    if param.attrib.get("direction") == "return":
                        type_elem = param.find("type", ns)
                        raw_type = ""
                        if type_elem is not None:
                            raw_type = type_elem.attrib.get('href', "") or type_elem.attrib.get('name', "")
                        else:
                            raw_type = param.attrib.get("type", "")
                        return_type = XMI2AST.normalize_type(raw_type)
                        if not return_type or len(return_type.strip())==0:
                            return_type = detected_return_type
                        break
                if not XMI2AST.is_camel_case(op_name) and len(op_name.strip()) > 0:
                    print("Error here method: " + op_name +" "+ repr(op_name)+ " file index: " + str(file_index))

                if len(params.strip())==0:
                    params="()"

                method_node = {
                    "type": "method",
                    "value": op_name + params,
                    "visibility": op_visibility,
                    "data_type": return_type
                }
                class_node["children"].append(method_node)

            classes.append(class_node)

        return classes

    @staticmethod
    def convertXMI(input_path: str, output_path: str, file_index):
        classes = XMI2AST.extract_classes(input_path, file_index)

        if not classes :
            return None, 0
        else:
            ast = {
                "type": "root",
                "children": classes
            }
            return ast, len(classes)


In [ ]:
import os

def convert_all_xmi_in_dir(input_dir: str, output_dir: str):
    count_of_classes = 0
    count_of_files = 0
    # Make sure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    file_index=-1
    # Loop through files in input_dir
    excluded=[1520]
    output_data = []
    for filename in os.listdir(input_dir):
        file_index=file_index+1
        if file_index < 0:
            continue

        if file_index in excluded:
            continue

        if filename.lower().endswith(".xmi"):
            input_path = os.path.join(input_dir, filename)
            # Replace .xmi extension with .json for output filename
            output_filename = os.path.splitext(filename)[0] + ".json"
            output_path = os.path.join(output_dir, output_filename)


            ast, len_classes = XMI2AST.convertXMI(input_path, output_path,file_index)

            if ast is None:
                continue
            count_of_classes=len_classes+count_of_classes
            count_of_files=count_of_files + 1
            output_data.append({"Output_AST": ast, "Model": output_filename})
            #print(f"Converting {input_path} -> {output_path}")
            #with open(output_path, 'w', encoding='utf-8') as out_file:
                #json.dump(ast, out_file, indent=2)
            print(f"Output saved to {output_path}")

    print(f"extracted classes count {count_of_classes}")
    print(f"files count {count_of_files}")
    with open("/content/drive/MyDrive/GenClass/data/raw/genmymodel_asts.json", "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2)

    with open("/content/drive/MyDrive/GenClass/data/raw/genmymodel_asts.jsonl", "w", encoding="utf-8") as f:
        for item in output_data:
            json.dump(item, f)
            f.write("\n")

# Example usage:
convert_all_xmi_in_dir("/content/drive/MyDrive/GenClass/temp/modelset/raw-data/repo-genmymodel-uml/data",
                       "/content/drive/MyDrive/GenClass/temp/modelset_AST_genmymodel")


# extracted classes count 32063
# files count 3085

In [ ]:
import xml.etree.ElementTree as ET
import json
import re
import urllib.parse
import unicodedata
import wordninja

class Ecore2AST:
    primitive_types = {"string", "boolean", "int", "float", "double", "long", "short", "byte", "char", "eint", "estring", "eboolean"}

    relation_symbols = {
        "generalization": "<|--",
        "implementation": "<|..",
        "composition": "*--",
        "aggregation": "o--",
        "dependency": "-->",
        "relation": "--"
    }

    @staticmethod
    def normalize_type(raw_type: str) -> str:
          if not raw_type:
                return ""

          raw_type = urllib.parse.unquote(raw_type).strip()

          if raw_type.startswith("_"):
                return ""

          if '#' in raw_type:
                raw_type = raw_type.split('#')[-1].lstrip('/')

          raw_type = Ecore2AST.strip_path_prefix(raw_type)
          lower_type = raw_type.lower()

          # Handle common synonyms and variations
          synonym_map = {
            "integer": "int",
            "int": "int",
            "eint": "int",
            "bool": "boolean",
            "boolean": "boolean",
            "eboolean": "boolean",
            "string": "string",
            "estring": "string",
            "float": "float",
            "double": "double",
            "long": "long",
            "short": "short",
            "byte": "byte",
            "char": "char"
          }

          if lower_type in synonym_map:
                return synonym_map[lower_type]

          return raw_type[0].upper() + raw_type[1:]


    @staticmethod
    def strip_path_prefix(s: str) -> str:
        return s.split("/")[-1] if "/" in s else s

    @staticmethod
    def to_pascal_case(name: str) -> str:
        if not name: return ""
        parts = re.split(r'[\s_-]+', name.strip())
        if len(parts) == 1:
            parts = wordninja.split(name)
        return ''.join(word.capitalize() for word in parts if word)

    @staticmethod
    def to_camel_case(name: str) -> str:
        if not name: return ""
        parts = re.split(r'[\s_-]+', name.strip())
        if len(parts) == 1 and parts[0].islower():
            parts = wordninja.split(name.strip())
        if not parts: return ""
        return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:])

    @staticmethod
    def remove_after_char(s: str, specific_char) -> str:
        return s.split(specific_char, 1)[0]

    @staticmethod
    def remove_accents(text: str) -> str:
        return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))

    @staticmethod
    def remove_after_brackets_and_parentheses(s: str) -> str:
        s = s.split('[')[0]
        s = s.split('(')[0]
        return s

    @staticmethod
    def replace_part(s: str, replaced: str, replacement: str = "") -> str:
        return s.replace(replaced, replacement)

    @staticmethod
    def extract_generic_type(s: str) -> str:
        match = re.search(r'<\s*([a-zA-Z0-9_]+)\s*>', s)
        return match.group(1) if match else s

    @staticmethod
    def extract_visibility(_) -> str:
        return "+"  # Ecore typically doesn't have visibility in XMI, default to public

    @staticmethod
    def split_method_and_return(sig: str) -> tuple[str, str]:
        """
        Given a signature like "getName():string",
        returns ("getName()", "string").
        If no return type is present, returns (sig, "").
        """
        sig = sig.strip()
        pattern = r'^(.+\(\))(?:\s*:\s*(.+))?$'
        match = re.match(pattern, sig)
        if match:
            method = match.group(1)
            ret_type = match.group(2) or ""
            return method, ret_type
        return sig, ""

    @staticmethod
    def split_method_signature(sig: str) -> tuple[str, str]:
        """
        Splits a method signature "name(params)" into ("name", "(params)").
        Returns (name, params_with_parentheses) or (sig, "") if no parentheses found.
        """
        sig = sig.strip()
        match = re.match(r'^([^(]+)(\(.+\))$', sig)
        if match:
            method_name = match.group(1).strip()
            params = match.group(2).strip()
            return method_name, params
        return sig, ""

    @staticmethod
    def manipulate_class_name(name):
        if not name: return ""
        name = Ecore2AST.remove_accents(name)
        name = Ecore2AST.extract_generic_type(name)
        name = Ecore2AST.remove_after_brackets_and_parentheses(name)
        name = Ecore2AST.remove_after_char(name, "/")
        name = Ecore2AST.remove_after_char(name, ":")
        name = Ecore2AST.remove_after_char(name, "=")
        name = Ecore2AST.replace_part(name, ".", "")
        name = Ecore2AST.replace_part(name, "+", "")
        name = Ecore2AST.strip_path_prefix(name)
        return Ecore2AST.to_pascal_case(name)

    @staticmethod
    def manipulate_attr_name(name):
        if not name: return ""
        name = Ecore2AST.extract_generic_type(name)
        name = Ecore2AST.remove_after_brackets_and_parentheses(name)
        name = Ecore2AST.remove_after_char(name, ".")
        name = Ecore2AST.remove_after_char(name, "=")
        name = Ecore2AST.remove_after_char(name, "*")
        name = Ecore2AST.replace_part(name, "+", "")
        name = Ecore2AST.replace_part(name, "#", "")
        name = Ecore2AST.replace_part(name, ":", "")
        name = Ecore2AST.replace_part(name, "$", "")
        name = Ecore2AST.replace_part(name, "'s", "")
        name = Ecore2AST.replace_part(name, "-", "")
        name = Ecore2AST.replace_part(name, "_", "")
        name = Ecore2AST.remove_accents(name)
        return Ecore2AST.to_camel_case(name.lower())

    @staticmethod
    def manipulate_method_name(op_name):
        if not op_name: return ""
        op_name = op_name.strip()
        op_name = Ecore2AST.remove_accents(op_name)
        op_name = Ecore2AST.remove_after_char(op_name, '.')
        op_name = Ecore2AST.remove_after_char(op_name, '(')
        op_name = Ecore2AST.replace_part(op_name, "void:", "")
        op_name = Ecore2AST.replace_part(op_name, "+void", "")
        op_name = Ecore2AST.replace_part(op_name, "+", "")
        op_name = Ecore2AST.replace_part(op_name, "_", "")
        op_name = Ecore2AST.replace_part(op_name, ":", "")
        op_name = Ecore2AST.replace_part(op_name, "'s", "")
        op_name = Ecore2AST.replace_part(op_name, "*", "")
        op_name = Ecore2AST.replace_part(op_name, "-", "")
        return Ecore2AST.to_camel_case(op_name.lower())




    @staticmethod
    def extract_classes_and_relations(ecore_file, file_index):
        def format_multiplicity(lower: str, upper: str) -> str:
            l = lower or "1"
            u = upper or "1"
            if u == "-1":
                u = "*"
            if l == "-1":
                l = "*"
            if l == u:
                return l
            return f"{l}..{u}"


        tree = ET.parse(ecore_file)
        root = tree.getroot()
        ns = {
            "xmi": "http://www.omg.org/XMI",
            "ecore": "http://www.eclipse.org/emf/2002/Ecore",
            "xsi": "http://www.w3.org/2001/XMLSchema-instance"
        }

        class_map = {}
        id_to_type = {}

        for classifier in root.findall(".//eClassifiers[@xsi:type='ecore:EClass']", ns):
            class_id = classifier.attrib.get("{http://www.omg.org/XMI}id")
            raw_name = classifier.attrib.get("name", "")
            class_name = Ecore2AST.manipulate_class_name(raw_name)
            class_name = Ecore2AST.strip_path_prefix(class_name)
            if class_id:
                class_map[class_id] = class_name
                id_to_type[class_id] = class_name

        ast_classes = []
        for class_elem in root.findall(".//eClassifiers[@xsi:type='ecore:EClass']", ns):
            raw_class_name = class_elem.attrib.get("name", "")
            class_name = Ecore2AST.manipulate_class_name(raw_class_name)
            class_name = Ecore2AST.strip_path_prefix(class_name)
            class_node = {
                "type": "class",
                "value": class_name,
                "children": []
            }

            # Attributes
            for attr in class_elem.findall("eStructuralFeatures[@xsi:type='ecore:EAttribute']", ns):
                  attr_name = Ecore2AST.manipulate_attr_name(attr.attrib.get("name", ""))
                  attr_type = Ecore2AST.normalize_type(attr.attrib.get("eType", ""))
                  attr_type = Ecore2AST.strip_path_prefix(attr_type)

                  # Extract visibility from annotations if available
                  visibility = "+"
                  for annotation in attr.findall("eAnnotations", ns):
                        if annotation.attrib.get("source", "").lower() == "visibility":
                              for detail in annotation.findall("details", ns):
                                    if detail.attrib.get("key") == "value":
                                          raw_visibility = detail.attrib.get("value", "+").lower()
                                          if raw_visibility in ["public", "+"]:
                                                visibility = "+"
                                          elif raw_visibility in ["private", "-"]:
                                                visibility = "-"
                                          elif raw_visibility in ["protected", "#"]:
                                                visibility = "#"
                                          elif raw_visibility in ["package", "~"]:
                                                visibility = "~"
                                          break

                  if attr_name:
                        class_node["children"].append({
                              "type": "attribute",
                              "value": attr_name,
                              "visibility": visibility,
                              "data_type": attr_type
                        })




            # Methods (Operations)
            for op in class_elem.findall("eOperations", ns):
                  raw_op_name = op.attrib.get("name", "")
                  op_name, detected_return_type = Ecore2AST.split_method_and_return(raw_op_name.strip())
                  op_name, params = Ecore2AST.split_method_signature(op_name.strip())
                  op_name = Ecore2AST.manipulate_method_name(op_name)

                  # Extract visibility from annotations if available
                  visibility = "+"
                  for annotation in op.findall("eAnnotations", ns):
                        if annotation.attrib.get("source", "").lower() == "visibility":
                              for detail in annotation.findall("details", ns):
                                    if detail.attrib.get("key") == "value":
                                          raw_visibility = detail.attrib.get("value", "+").lower()
                                          if raw_visibility in ["public", "+"]:
                                                visibility = "+"
                                          elif raw_visibility in ["private", "-"]:
                                                visibility = "-"
                                          elif raw_visibility in ["protected", "#"]:
                                                visibility = "#"
                                          elif raw_visibility in ["package", "~"]:
                                                visibility = "~"
                                          break

                  # Get return type from operation
                  return_type = "void"
                  for param in op.findall("eParameters", ns):
                        if param.attrib.get("direction", "").lower() == "return":
                              raw_type = param.attrib.get("eType", "")
                              if raw_type.startswith("#//"):
                                    type_id = raw_type[3:]
                                    return_type = class_map.get(type_id, Ecore2AST.normalize_type(raw_type))
                              else:
                                    return_type = Ecore2AST.normalize_type(raw_type)
                              break

                  if not return_type:
                        return_type = detected_return_type

                  method_node = {
                        "type": "method",
                        "value": op_name + "()",
                        "visibility": visibility,
                        "data_type": return_type
                  }
                  class_node["children"].append(method_node)

            # References
            for ref in class_elem.findall("eStructuralFeatures[@xsi:type='ecore:EReference']", ns):
                ref_name = Ecore2AST.manipulate_attr_name(ref.attrib.get("name", ""))
                ref_type_raw = ref.attrib.get("eType", "")
                ref_type_id = ""
                if ref_type_raw.startswith("#//"):
                    ref_type_id = ref_type_raw[3:]
                ref_type = class_map.get(ref_type_id, Ecore2AST.normalize_type(ref_type_raw))
                ref_type = Ecore2AST.strip_path_prefix(ref_type)

                containment = ref.attrib.get("containment", "false").lower() == "true"
                aggregation = ref.attrib.get("aggregation", "").lower()
                derived = ref.attrib.get("derived", "false").lower() == "true"

                """
                if ref_name:
                    class_node["children"].append({
                        "type": "attribute",
                        "value": ref_name,
                        "visibility": "+",
                        "data_type": ref_type
                    })
                """

                if ref_type and ref_type != class_name:
                    rel_type = "--"
                    if containment or aggregation == "composite":
                        rel_type = "*--"
                    elif aggregation == "shared":
                        rel_type = "o--"
                    else:
                        rel_type = "-->"  # default: plain association

                    multiplicity1 = format_multiplicity(ref.attrib.get("lowerBound"), ref.attrib.get("upperBound"))

                    # Resolve multiplicity2 using eOpposite if available
                    multiplicity2 = "1"  # default fallback
                    e_opposite = ref.attrib.get("eOpposite")
                    if e_opposite:
                        e_opposite_id = e_opposite.split("/")[-1]
                        for opp in root.findall(".//eStructuralFeatures[@xsi:type='ecore:EReference']", ns):
                            opp_id = opp.attrib.get("{http://www.omg.org/XMI}id")
                            if opp_id and opp_id.endswith(e_opposite_id):
                                opp_lower = opp.attrib.get("lowerBound")
                                opp_upper = opp.attrib.get("upperBound")
                                multiplicity2 = format_multiplicity(opp_lower, opp_upper)
                                break

                    relation_node = {
                        "type": "relation",
                        "value": rel_type,
                        "multiplicity1": multiplicity1,
                        "multiplicity2": multiplicity2,
                        "children": [
                            {"type": "class", "value": ref_type}
                        ]
                    }
                    label = ref.attrib.get("name", "")
                    if label:
                        relation_node["label"] = label
                    class_node["children"].append(relation_node)



            # Super types
            super_types = class_elem.attrib.get("eSuperTypes", "")
            for super_ref in super_types.split():
                if super_ref.startswith("#//"):
                    super_class_id = super_ref[3:]
                    super_elem = class_map.get(super_class_id)

                    if super_elem is not None:
                        is_interface = super_elem.attrib.get("interface", "false").lower() == "true"
                        is_abstract = super_elem.attrib.get("abstract", "false").lower() == "true"

                        super_class_name = Ecore2AST.manipulate_class_name(super_class_id)
                        super_class_name = Ecore2AST.strip_path_prefix(super_class_name)

                        rel_value = "..|>" if is_interface else "--|>"

                        generalization_relation = {
                            "type": "relation",
                            "value": rel_value,
                            "multiplicity1": "1",
                            "multiplicity2": "1",
                            "label": "",
                            "children": [
                                {"type": "class", "value": super_class_name}
                            ]
                        }
                        if not super_class_name=="AbstractElement":
                            class_node["children"].append(generalization_relation)

            ast_classes.append(class_node)
        return ast_classes

    @staticmethod
    def convertEcore(input_path: str, output_path: str, file_index):
        elements = Ecore2AST.extract_classes_and_relations(input_path, file_index)

        if not elements:
            return None, 0
        else:
            ast = {
                "type": "root",
                "children": elements
            }
            if output_path:
                with open(output_path, 'w') as f:
                    json.dump(ast, f, indent=2)
            return ast, len(elements)

In [ ]:
import os
import json
from pathlib import Path

def find_all_ecore_files(input_dir: str):
    ecore_files = []
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(".ecore"):
                full_path = os.path.join(root, file)
                ecore_files.append(full_path)
    return ecore_files

def convert_all_ecore_files(ecore_files,input_dir: str, output_dir: str):
    count_of_classes = 0
    count_of_files = 0
    file_index = -1
    excluded = [236]  # Customize this as needed

    os.makedirs(output_dir, exist_ok=True)
    output_data=[]
    for filename in ecore_files:
        file_index += 1

        if file_index < 0:
            continue

        if file_index in excluded:
            continue

        if filename.lower().endswith(".ecore"):
            input_path = os.path.join(input_dir, filename)
            output_filename = os.path.splitext(filename)[0] + ".json"
            output_path = os.path.join(output_dir, f"{file_index}.json")

            ast, len_classes  = Ecore2AST.convertEcore(input_path,output_path,file_index)

            if ast is None:
                #print(f"⚠️  Skipped (no classes): {input_path}")
                continue


            count_of_classes += len_classes
            count_of_files += 1
            output_data.append({"Output_AST": ast, "Model": output_filename})
            #with open(output_path, 'w', encoding='utf-8') as out_file:
                #json.dump(ast, out_file, indent=2)

            print(f"✅ Converted [{file_index}] {input_path} → {output_path} ({len_classes} classes)")

    print(f"📦 Total extracted classes: {count_of_classes}")
    print(f"📁 Total files converted: {count_of_files}")
    with open("/content/drive/MyDrive/GenClass/data/raw/ecore_asts.json", "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2)

    with open("/content/drive/MyDrive/GenClass/data/raw/ecore_asts.jsonl", "w", encoding="utf-8") as f:
        for item in output_data:
            json.dump(item, f)
            f.write("\n")


# Example usage
input_dir = "/content/drive/MyDrive/GenClass/temp/modelset/raw-data/repo-ecore-all"
output_dir = "/content/drive/MyDrive/GenClass/temp/modelset_AST_ecore_no_empty"


file_path = "/content/drive/MyDrive/GenClass/temp/ecore_files.txt"

if not os.path.exists(file_path):
    ecore_paths = find_all_ecore_files(input_dir)
    print(f"🔍 Found {len(ecore_paths)} .ecore files.")

    with open(file_path, 'w', encoding='utf-8') as f:
        for path in ecore_paths:
            f.write(path + '\n')
else:
    print("✅ File already exists, skipping scan.")



with open(file_path, 'r', encoding='utf-8') as f:
    ecore_paths = [line.strip() for line in f if line.strip()]

convert_all_ecore_files(ecore_paths, input_dir, output_dir)
# Total extracted classes: 140619
# Total files converted: 5432

In [ ]:
import random
with open("/content/drive/MyDrive/GenClass/temp/ecore_files.txt", 'r', encoding='utf-8') as f:
    ecore_paths = [line.strip() for line in f if line.strip()]

#input_path =ecore_paths[108]
input_path =random.choice(ecore_paths)
input_path ="/content/drive/MyDrive/GenClass/temp/modelset/raw-data/repo-ecore-all/data/abreslav/astrans/ru.ifmo.rain.astrans.interpreter.trace/model/trace.ecore"
ast, len_classes = Ecore2AST.convertEcore(input_path,"fdsf",1)
print(json.dumps(ast, indent=2))

In [ ]:
import wordninja
import re
class NameFormatter:

    @staticmethod
    def to_camel_case_attr(name: str) -> str:
        if not name:
            return ""

        # First try splitting by delimiters (_ - space)
        parts = re.split(r'[\s_-]+', name.strip())

        # If splitting fails (i.e., only one part and all lowercase), use wordninja
        if len(parts) == 1 and parts[0].islower():
            parts = wordninja.split(name.strip())

        if not parts:
            return ""

        return parts[0].lower() + ''.join(word.capitalize() for word in parts[1:])

    @staticmethod
    def to_camel_case_method(name: str) -> str:
        if not name:
            return ""

        if '(' in name:
            method_name, rest = name.split('(', 1)
            camel_name = NameFormatter.to_camel_case_attr(method_name.strip())
            return f"{camel_name}({rest}"
        else:
            # No parentheses, treat whole string as method name
            return NameFormatter.to_camel_case_attr(name.strip())


    @staticmethod
    def to_pascal_case(name: str) -> str:
        if not name:
            return ""

        # First split on delimiters like space, underscore, dash
        parts = re.split(r'[\s_-]+', name.strip())

        # If splitting didn't produce multiple parts, use wordninja
        if len(parts) == 1:
            parts = wordninja.split(name)

        return ''.join(word.capitalize() for word in parts if word)


    @staticmethod
    def lowercase_first_letter(s: str) -> str:
        return s[:1].lower() + s[1:] if s else ""

    @staticmethod
    def uppercase_first_letter(s: str) -> str:
        return s[:1].upper() + s[1:] if s else ""


a = NameFormatter.to_camel_case_method("publicoperation")
print(a)

publicOperation
